# Feature extraction
Given that we have transformed the time-dependent signal into the frequency domain, we can extract features from it.

In [27]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
from typing import Tuple
from ipywidgets import widgets
ws = {'description_width': 'initial'}


def generate_signal_and_fft(T:float, fA:int, fB:int, use_hamming:bool, noisy:bool, sampling_rate:int):
    t = np.linspace(0, T, int(T*sampling_rate), endpoint=False)
    N = len(t)
    signal = 2.3 * np.cos(2*np.pi*fA*t) + 4 * np.sin(2*np.pi*fB*t)
    hamming = np.hamming(N) if use_hamming else np.ones(N)
    noise = np.random.randn(N) if noisy else 0
    noisy_signal = (signal + noise)*hamming
    fft = np.fft.rfft(noisy_signal)
    frequencies = np.fft.rfftfreq(len(noisy_signal), d=1./sampling_rate)
    return (t, signal), (frequencies, fft)

We can now extract some features from this plot, like the expected value $E[\vert\hat x\vert]=\frac{1}{N}\sum_{n=0}^{N-1} \vert \hat x_n \vert$ of the amplitude, $y=\vert \hat x \vert$, $y_n=\vert \hat x_n \vert$.

In [28]:
(t, signal), (frequencies, fft) = generate_signal_and_fft(T=10, fA=1, fB=3, 
                                                          use_hamming=True, noisy=False, sampling_rate=32)
y = np.abs(fft)
mu = np.sum(y)/len(y)

The variance of the signal is $\mathrm{Var}[y]=E[(y - E(y))^2]=\frac{1}{N}\sum_{n=0}^{N-1} \left( \hat y_n - \frac{1}{N}\sum_{n=0}^{N-1} y_n \right)^2$


In [29]:
sigma = np.sum((y -mu)**2)/len(y)

We print the two features

In [30]:
print(f"Expected frequency amplitude {mu:.2e}, frequency variance {sigma:.2e}")

Expected frequency amplitude 6.27e+00, frequency variance 1.30e+03


We can also compute the RMS value of the Power Spectral density for $N$ number of bands

In [37]:
@widgets.interact(
    use_hamming=widgets.Checkbox(value=True, description="Hamming"),
    noisy_signal=widgets.Checkbox(value=True, description="Noise"),
    sampling_rate=widgets.ToggleButtons(options=[20000, 40000],description="Sampling rate (Hz)", style=ws),
    fA=widgets.Select(options=[100,1000,5000], value=100, description='fA', disabled=False),
    fB=widgets.Select(options=[500,8000,9000], value=8000, description='fB', disabled=False),
    N=widgets.IntSlider(4, min=1, max=33, description="Number of bins",style=ws))
def RMS_PSD(sampling_rate:int, use_hamming:bool, noisy_signal:bool, N:int, fA:int, fB:int)-> Tuple[npt.NDArray[np.int32], npt.NDArray[np.float64]]:
    """
    Compute the root mean square of the power spectral density of 
    a discrete Fourier series by subdividing the frequencies
    into `N` number of bins
    """
    T = 1
    (t, signal), (frequencies, fft) = generate_signal_and_fft(T, fA, fB, use_hamming, noisy_signal, sampling_rate)
    if N > len(fft):
        print(f"Number of windows cannot be more than {sampling_rate//2+1}. Currently {N}, set to {sampling_rate//2+1}")
        N = sampling_rate//2+1
    df = frequencies[1] - frequencies[0]
    f_per_band = len(fft)//N
    rem = len(fft)%N
    freq_per_band = np.zeros(N, dtype=np.int32)
    freq_per_band[:] = f_per_band
    freq_per_band[:rem]+=1
    offsets= np.zeros(N+1, dtype=np.int32)
    offsets[1:] = np.cumsum(freq_per_band)
    RMS = np.zeros(N, dtype=np.float64)
    mid = np.zeros(N, dtype=np.int32)
    width = np.zeros(N, dtype=np.float64)
    for i in range(N):
        mid[i] = offsets[i] + int((offsets[i+1]-offsets[i])/2)
        width[i] = df*freq_per_band[i]
        psd = np.abs(fft[offsets[i]:offsets[i+1]])**2
        RMS[i] = np.sqrt(1/len(signal)*np.sum(psd**2))
    
    fig = plt.figure()
    plt.title(f"Root mean square of Power Spectral Density with {N} bins")
    plt.grid()
    plt.bar(frequencies[mid], RMS, width=width)
    plt.gca().set_yscale('log')
    fig2 = plt.figure()
    plt.title("Frequency Amplitude")
    plt.plot(frequencies, np.abs(fft))


interactive(children=(ToggleButtons(description='Sampling rate (Hz)', options=(20000, 40000), style=ToggleButt…